In [ ]:
!pip install selenium

In [ ]:

# Import required modules



from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Set up WebDriver options
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)

# Define the URL
url="https://www.google.com/maps/place/Oakridge+International+School,+Bachupally/@17.5629181,78.3824724,17z/data=!4m8!3m7!1s0x3bcb8dc64ffffff7:0x8156e1ed344f53bb!8m2!3d17.5629181!4d78.3850473!9m1!1b1!16s%2Fg%2F11b637mfy7?entry=ttu"
# Initialize WebDriver
driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.get(url)
time.sleep(2)

# Scroll to load all reviews
scrollable_div = driver.find_element(By.XPATH, value='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
total_number_of_reviews = driver.find_element(By.XPATH, value='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[3]').text.split(" ")[0]
total_number_of_reviews = int(total_number_of_reviews.replace(',', '')) if ',' in total_number_of_reviews else int(total_number_of_reviews)

for i in range(0, (round(total_number_of_reviews / 10 - 1))):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
    time.sleep(1.5)

# Wait for the page to load (adjust the sleep time as necessary)
time.sleep(5)

# Find and click all "More" buttons
more_buttons = driver.find_elements(By.XPATH, "//button[contains(@class, 'w8nwRe kyuRq') and contains(@aria-label, 'See more')]")

for button in more_buttons:
    try:
        # Click the button
        driver.execute_script("arguments[0].click();", button)
        # Wait a bit for the action to complete
        time.sleep(1)
    except Exception as e:
        # Pass to continue even if there's an exception
        pass

# Add additional wait to ensure text is expanded
time.sleep(5)

# Get the page source
html_content = driver.page_source

# Close the driver
driver.quit()



In [ ]:
with open('school_review.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

In [ ]:
with open('school_review.html','r',encoding='utf-8') as f:
    html = f.read()

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [ ]:
soup = BeautifulSoup(html,'lxml')

In [ ]:
print(soup.prettify())

In [ ]:
containers = soup.find_all('div',class_='jftiEf fontBodyMedium')

In [ ]:
len(containers)

In [ ]:
print(len(containers))

In [ ]:

user_review_no=[]
rating=[]
time_ago=[]
comment_likes=[]
user_text=[]

for i in containers:
 
    try:
        user_review_no.append(i.find('div',class_='RfnDt').text.split('·')[1])
    except:
        user_review_no.append(np.nan)
        
    try:
        rating.append(i.find('span',class_='kvMYJc')["aria-label"])
    except:
        rating.append(np.nan)
        
    try:
        time_ago.append(i.find('span',class_='rsqaWe').text)
    except:
        time_ago.append(np.nan)
        
    try:
        comment_likes.append(i.find('span',class_='pkWtMe').text)
    except:
        comment_likes.append(np.nan)
        
    try:
        user_text.append(i.find('span',class_='wiI7pd').text)
    except:
        user_text.append(np.nan)

In [ ]:
#containers[0].find('div',class_='RfnDt').text.split('·')[1]

In [ ]:
#rsqaWe.find('span',class_='kvMYJc')["aria-label"]

In [ ]:
#containers[0].find('span',class_='kvMYJc')["aria-label"]

In [ ]:
#containers[0].find('span',class_='rsqaWe').text

In [ ]:
#containers[0].find('span',class_='wiI7pd').text

In [ ]:
#containers.find('div',class_='RfnDt').text.split('·')[1]

In [ ]:
df = pd.DataFrame({
'user_review_no' :user_review_no,
'rating' :rating,
'time_ago' :time_ago,
'comment_likes':comment_likes,
'user_text':user_text})


In [ ]:
df.head()

In [ ]:
from IPython.display import HTML
import base64  
import pandas as pd  

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(index =False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df)